In [1]:
import plotly
import plotly.graph_objs as go
from IPython.display import display, HTML
plotly.offline.init_notebook_mode(connected=True)

In [2]:
from rainman2 import RAINMAN2

Rainman2's logging has been configured!


Using TensorFlow backend.



#### Normal case: Start with 30% Random actions (explore) and rest 70% fetch max actions (exploit)


In [4]:
# Server profile: num_ues=200, APs=16, Scale=200.0, explore_radius=1
from collections import OrderedDict
ALGORITHM_CONFIG = OrderedDict(
    EPISODES=400,
    ALPHA=0.2,
    GAMMA=0.9,
    EPSILON=0.3,
    EPSILON_DECAY=0.99,
    EPSILON_MIN=0.01,
    VERBOSE=True,
    L1_HIDDEN_UNITS=13,
    L2_HIDDEN_UNITS=13,
    L1_ACTIVATION='relu',
    L2_ACTIVATION='relu',
    LOSS_FUNCTION='mean_squared_error',
    OPTIMIZER='Adam',
)

CELLULAR_MODEL_CONFIG = OrderedDict(
    NAME='Cellular',
    TYPE='Dev',
    SERVER='0.0.0.0',
    SERVER_PORT='8000',
    VERBOSE=True,
)


In [5]:
RAINMAN2.algorithm_config = ALGORITHM_CONFIG
RAINMAN2.environment_config = CELLULAR_MODEL_CONFIG

In [6]:
result = RAINMAN2.run_experiment("Cellular", "Qlearning", "Naive")

2018-05-08 10:56:03,913 Aris-MacBook-Pro.local Rainman2[22317] INFO Starting experiment!
2018-05-08 10:56:03,915 Aris-MacBook-Pro.local Rainman2[22317] INFO Building Environment instance: Cellular
2018-05-08 10:56:03,916 Aris-MacBook-Pro.local rainman2.lib.environment.cellular.base[22317] INFO Instantiating Cellular client: Dev
2018-05-08 10:56:03,917 Aris-MacBook-Pro.local QCellularAgent[22317] INFO Configuration used for the QCellular Agent:
2018-05-08 10:56:03,918 Aris-MacBook-Pro.local QCellularAgent[22317] INFO episodes: 400
2018-05-08 10:56:03,919 Aris-MacBook-Pro.local QCellularAgent[22317] INFO alpha: 0.2
2018-05-08 10:56:03,920 Aris-MacBook-Pro.local QCellularAgent[22317] INFO gamma: 0.9
2018-05-08 10:56:03,921 Aris-MacBook-Pro.local QCellularAgent[22317] INFO epsilon: 0.3
2018-05-08 10:56:03,922 Aris-MacBook-Pro.local QCellularAgent[22317] INFO epsilon_decay: 0.99
2018-05-08 10:56:03,923 Aris-MacBook-Pro.local QCellularAgent[22317] INFO epsilon_min: 0.01
2018-05-08 10:56:03,9

Module: run_experiment took: 203.312857s


[========================================================================] 100%


In [7]:
print("Number of states encountered: {}".format(len(result.Q)))

Number of states encountered: 2662


In [8]:
print("Number of q_ap_states encountered: {}".format(len(result.Q_ap)))

Number of q_ap_states encountered: 849


In [ ]:
print(result.Q)

In [ ]:
print(result.Q_ap)

In [ ]:
print(result.UE_AP_LIST)

In [10]:
# Show distribution of UEs and APs
ue_ap_list = result.UE_AP_LIST[0]
nodes = [(0, 0)]
nodes += [tuple(x.location) for x in ue_ap_list]

edges = []
edge_color = []
# edge_color = ['green'] * (len(edges))

# add edges between UE and AP
for i, ue in enumerate(ue_ap_list[16:]):
    if ue.sla == 1:
        color = "green"
    else:
        color = "red"
    edges.append((ue.ap, i+17))
    edge_color.append(color)

    
nodes_color = []
nodes_color += ['orange'] * 16

for ue in ue_ap_list[16:]:
    if ue.app == 2:
        # video UEs
        color = 'blue'
    else:
        # Web UEs
        color = 'magenta'
    nodes_color.append(color)




n_x, n_y = zip(*nodes[1:])
nodes_dict=dict(type='scatter',
           x=n_x,
           y=n_y,
           mode='markers',
           marker=dict(size=8, color=nodes_color))

edges_list = []
for k, e in enumerate(edges):
    info = dict(
        type='scatter',
        x=[nodes[e[0]][0], nodes[e[1]][0]],
        y=[nodes[e[0]][1], nodes[e[1]][1]],
        mode='lines',
        line=dict(width=2, color=edge_color[k])
    )
    edges_list.append(info)

data=edges_list+[nodes_dict]

layout_dict = dict(xaxis={'range': [0, 1600], 'title': 'x'},
              yaxis={'range': [0, 1600], 'title': 'y'},
              title="UE and AP Distribution in the grid",
              height=600,
              width=800,
              showlegend=False
              )
layout = go.Layout(layout_dict)

fig=dict(data=data, layout=layout)

plotly.offline.iplot(fig)

In [11]:
# Create rewards graph
rewards = go.Scatter(
    x = len(result.Rewards),
    y = result.Rewards,
    name='Rewards'
)

data = [rewards]


# Plot and embed in ipython notebook!
plotly.offline.iplot(
    {"data": data,
     "layout": go.Layout(
         height=500,
         width=800,
         title="Rewards over time",
         xaxis=dict(
             title='Episodes',
         ),
         yaxis=dict(
             title='Rewards',
         ), 
         showlegend=True)},
    show_link=False,
    link_text=None)



In [12]:
# Create handoff graph
handoffs = go.Scatter(
    x = len(result.Handoffs),
    y = result.Handoffs,
    name='Handoffs'
)

data = [handoffs]

# Plot and embed in ipython notebook!
plotly.offline.iplot(
    {"data": data,
     "layout": go.Layout(
         height=500,
         width=800,
         title='Handoffs over time',
         xaxis=dict(
             title='Episodes',
         ),
         yaxis=dict(
             title='Handoffs'
         ), 
         showlegend=True)},
    show_link=False,
    link_text=None)


In [13]:
# Create UE graph
ue_sla = go.Scatter(
    x = len(result.Meets_SLA),
    y = result.Meets_SLA,
    name='UE SLA'
)

data = [ue_sla]

# Plot and embed in ipython notebook!
plotly.offline.iplot(
    {"data": data,
     "layout": go.Layout(
         height=500,
         width=800,
         title="UE's SLA over time",
         xaxis=dict(
             title='Episodes',
         ),
         yaxis=dict(
             title='UE SLA'
         ), 
         showlegend=True)},
    show_link=False,
    link_text=None)